In [1]:
import keras
keras.__version__

Using TensorFlow backend.
/home/sasaki/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sasaki/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sasaki/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/sasaki/.local/lib/python3.6/site-packages/tensorflow/python

'2.3.1'

In [2]:
from keras.applications import inception_v3
from keras import backend as K

# We will not be training our model,
# so we use this command to disable all training-specific operations
K.set_learning_phase(0)

# Build the InceptionV3 network.
# The model will be loaded with pre-trained ImageNet weights.
model = inception_v3.InceptionV3(weights='imagenet',
                                 include_top=False)

In [3]:
# Dict mapping layer names to a coefficient
# quantifying how much the layer's activation
# will contribute to the loss we will seek to maximize.
# Note that these are layer names as they appear
# in the built-in InceptionV3 application.
# You can list all layer names using `model.summary()`.
layer_contributions = {
    'mixed2': 0.2,
    'mixed3': 3.,
    'mixed4': 2.,
    'mixed5': 1.5,
}

In [4]:
# Get the symbolic outputs of each "key" layer (we gave them unique names).
layer_dict = dict([(layer.name, layer) for layer in model.layers])

# Define the loss.
loss = K.variable(0.)
for layer_name in layer_contributions:
    # Add the L2 norm of the features of a layer to the loss.
    coeff = layer_contributions[layer_name]
    activation = layer_dict[layer_name].output

    # We avoid border artifacts by only involving non-border pixels in the loss.
    scaling = K.prod(K.cast(K.shape(activation), 'float32'))
    # 2020.02.26 bug-fix: Variable += value not supported.
    #loss += coeff * K.sum(K.square(activation[:, 2: -2, 2: -2, :])) / scaling
    #loss.assign_add(coeff * K.sum(K.square(activation[:, 2: -2, 2: -2, :])) / scaling)
    loss = loss + coeff * K.sum(K.square(activation[:, 2: -2, 2: -2, :])) / scaling

In [5]:
# This holds our generated image
dream = model.input

# Compute the gradients of the dream with regard to the loss.
# 2020.02.29 change
grads = K.gradients(loss, dream)[0]
#grads = K.gradients(loss, [dream])[0]
print('grads=', grads)

# Normalize gradients.
# 2020.02.26 delete:
# ValueError: None values not supported.
#grads /= K.maximum(K.mean(K.abs(grads)), 1e-7)

# Set up function to retrieve the value
# of the loss and gradients given an input image.
outputs = [loss, grads]
# 2020.02.29 add
print(dream.name)
print(loss.name)
print(grads.name)
#setattr(dream, 'name', 'dream')
#setattr(loss, 'name', 'loss')
#setattr(grads, 'name', 'grads')

# 2020.02.29 change
# AttributeError: 'NoneType' object has no attribute 'name'
fetch_loss_and_grads = K.function([dream], outputs)
#fetch_loss_and_grads = K.function([dream], outputs, name='loss_grads')

def eval_loss_and_grads(x):
    outs = fetch_loss_and_grads([x])
    loss_value = outs[0]
    grad_values = outs[1]
    return loss_value, grad_values

def gradient_ascent(x, iterations, step, max_loss=None):
    for i in range(iterations):
        loss_value, grad_values = eval_loss_and_grads(x)
        if max_loss is not None and loss_value > max_loss:
            break
        print('...Loss value at', i, ':', loss_value)
        x += step * grad_values
    return x

grads= Tensor("gradients/conv2d_1/convolution_grad/Conv2DBackpropInput:0", shape=(?, ?, ?, 3), dtype=float32)
input_1:0
add_3:0
gradients/conv2d_1/convolution_grad/Conv2DBackpropInput:0



In [8]:
import scipy
# 2020.02.29 add
# change scipy.image.imread/imsave -> imageio.imread/imwrite
import imageio
from keras.preprocessing import image

def resize_img(img, size):
    img = np.copy(img)
    factors = (1,
               float(size[0]) / img.shape[1],
               float(size[1]) / img.shape[2],
               1)
    return scipy.ndimage.zoom(img, factors, order=1)


def save_img(img, fname):
    pil_img = deprocess_image(np.copy(img))
    # 2020.02.29 change
    #scipy.misc.imsave(fname, pil_img)
    imageio.imwrite(fname, pil_img)


def preprocess_image(image_path):
    # Util function to open, resize and format pictures
    # into appropriate tensors.
    img = image.load_img(image_path)
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = inception_v3.preprocess_input(img)
    return img


def deprocess_image(x):
    # Util function to convert a tensor into a valid image.
    if K.image_data_format() == 'channels_first':
        x = x.reshape((3, x.shape[2], x.shape[3]))
        x = x.transpose((1, 2, 0))
    else:
        x = x.reshape((x.shape[1], x.shape[2], 3))
    x /= 2.
    x += 0.5
    x *= 255.
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [9]:
import numpy as np

# Playing with these hyperparameters will also allow you to achieve new effects

step = 0.01  # Gradient ascent step size
num_octave = 3  # Number of scales at which to run gradient ascent
octave_scale = 1.4  # Size ratio between scales
iterations = 20  # Number of ascent steps per scale

# If our loss gets larger than 10,
# we will interrupt the gradient ascent process, to avoid ugly artifacts
max_loss = 10.

# Fill this to the path to the image you want to use
#base_image_path = '/home/ubuntu/data/original_photo_deep_dream.jpg'
base_image_path = '/home/sasaki/Pictures/sample1_l.jpg'

# Load the image into a Numpy array
img = preprocess_image(base_image_path)

# We prepare a list of shape tuples
# defining the different scales at which we will run gradient ascent
original_shape = img.shape[1:3]
successive_shapes = [original_shape]
for i in range(1, num_octave):
    shape = tuple([int(dim / (octave_scale ** i)) for dim in original_shape])
    successive_shapes.append(shape)

# Reverse list of shapes, so that they are in increasing order
successive_shapes = successive_shapes[::-1]

# Resize the Numpy array of the image to our smallest scale
original_img = np.copy(img)
shrunk_original_img = resize_img(img, successive_shapes[0])

for shape in successive_shapes:
    print('Processing image shape', shape)
    img = resize_img(img, shape)
    img = gradient_ascent(img,
                          iterations=iterations,
                          step=step,
                          max_loss=max_loss)
    upscaled_shrunk_original_img = resize_img(shrunk_original_img, shape)
    same_size_original = resize_img(original_img, shape)
    lost_detail = same_size_original - upscaled_shrunk_original_img

    img += lost_detail
    shrunk_original_img = resize_img(original_img, shape)
    save_img(img, fname='dream_at_scale_' + str(shape) + '.png')

save_img(img, fname='final_dream.png')

Processing image shape (365, 242)
...Loss value at 0 : 1.666436
...Loss value at 1 : 1.6678383
...Loss value at 2 : 1.6692272
...Loss value at 3 : 1.6706098
...Loss value at 4 : 1.6719788
...Loss value at 5 : 1.6733398
...Loss value at 6 : 1.6747074
...Loss value at 7 : 1.6760739
...Loss value at 8 : 1.677438
...Loss value at 9 : 1.6787956
...Loss value at 10 : 1.6801574
...Loss value at 11 : 1.6815124
...Loss value at 12 : 1.6828678
...Loss value at 13 : 1.6842253
...Loss value at 14 : 1.6855745
...Loss value at 15 : 1.6869202
...Loss value at 16 : 1.6882678
...Loss value at 17 : 1.6896117
...Loss value at 18 : 1.6909363
...Loss value at 19 : 1.6922584
Processing image shape (512, 340)
...Loss value at 0 : 1.5539804
...Loss value at 1 : 1.5549282
...Loss value at 2 : 1.5558814
...Loss value at 3 : 1.5568389
...Loss value at 4 : 1.5577935
...Loss value at 5 : 1.5587449
...Loss value at 6 : 1.5596935
...Loss value at 7 : 1.5606341
...Loss value at 8 : 1.5615721
...Loss value at 9 : 1.56